In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import socket
import sys
import requests
import requests_oauthlib
import json
import pandas as pd
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
analyser = SentimentIntensityAnalyzer()

# Replace the values below with yours
# Replace the values below with yours
ACCESS_TOKEN = 'YOUR ACCESS TOKEN'
ACCESS_SECRET = 'YOUR ACCESS SECRET'
CONSUMER_KEY = 'YOUR CONSUMER KEY'
CONSUMER_SECRET = 'YOUR CONSUMER SECRET'
my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)
#my_auth = requests_oauthlib.OAuth1(CONSUMER_KEY, CONSUMER_SECRET,ACCESS_TOKEN, ACCESS_SECRET)

list2=[]
def trending_hash_tag(row):
    global list2
    C = Counter(list2)
    string = str(row["Text"])
    tag = re.findall(r"#(\w+)", string)
    dict1={}
    if len(tag)>0:
        for element in tag:
            dict1[element]= C[element]
        trending_hashtag= sorted(dict1)[0]
    else:
        trending_hashtag=""
    return trending_hashtag
    
list1 =[]
def hash_tag(row):
    global list1
    string = str(row["Text"])
    tag = re.findall(r"#(\w+)", string)
    list1.append(tag) 
    return " ".join(tag)
tweet_list =[]
def save_tweet(tweet):
    global tweet_list
    tweet = tweet.split("*////*")
    score = analyser.polarity_scores(tweet[4])["compound"]
    if score >0.33:
        sentiment = "positive"
    elif score <-0.33:
        sentiment = "negative"
    else:
        sentiment = "neutral"
    tweet.append(sentiment)
    tweet_list = tweet_list+[tweet]
    data_frame =pd.DataFrame(tweet_list,columns=['Time','longitude', 'latitude','place', "Text", "Sentiment"])
    data_frame["hashstag"] = data_frame.apply(hash_tag,axis =1)
    data_frame["Trending hashtag"]= data_frame.apply(trending_hash_tag,axis=1)
    #print(tweet)
    #data_frame.append(tweet, ignore_index = True)
    data_frame.to_csv("Tweet_data.csv")
    
    
    

def send_tweets_to_spark(conn,http_resp):
    for line in http_resp.iter_lines():
        try:
            #conn, addr = new_socket.accept()
            full_tweet = json.loads(line)
            # refer https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html for details
            tweet_text = str(full_tweet["created_at"])+"*////*"+str(full_tweet["coordinates"]["coordinates"][0])+"*////*"+str(full_tweet["coordinates"]["coordinates"][1])+"*////*"+full_tweet["place"]["name"]+"*////*"+full_tweet["text"]
            #tweet_text  = full_tweet['text']
            #row = [ user, text, latitude, longitude, gender ]
            data = tweet_text+'\n'
            #save_tweet(tweet_text)
            #print("Tweet Text: " + tweet_text)
            #print ("------------------------------------------")
            conn.send(data.encode('ascii','ignore'))
            #conn.close()
        except:
            e = sys.exc_info()[0]
            #print("Error: %s" % e)


def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-180,-60,180,60'),('track','#')]
    #query_data = [('language', 'en')]
    #query_data = [('locations', '-130,-20,100,50'), ('track', '#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response


TCP_IP = "localhost"
TCP_PORT = 5000
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected to :",addr)
resp = get_tweets()
print("Starting getting tweets.")
send_tweets_to_spark(conn,resp)

Waiting for TCP connection...
Connected to : ('127.0.0.1', 60344)
https://stream.twitter.com/1.1/statuses/filter.json?language=en&locations=-180,-60,180,60&track=# <Response [200]>
Starting getting tweets.
